## Install Dependencies (if needed) ## 

In [ ]:
""" !pip install statsmodel
!pip install pyspark
!pip install pandas
!pip install plotly
!pip install geopandas
!pip install numpy
!pip install sklearn """


## Create Spark Session ##

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import plotly.express as px
import pandas as pd
import geopandas as gpd

spark = SparkSession.builder \
    .appName("NYC Taxi — Analysis Only") \
    .config("spark.driver.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "50") \
    .getOrCreate()

print("Spark session created.")

## Load Data from Parquet ##

In [ ]:
combined_df = spark.read.parquet("Cleaned_Parquet/combined/")
city_hourly = spark.read.parquet("Cleaned_Parquet/city_hourly/")
zone_hourly = spark.read.parquet("Cleaned_Parquet/zone_hourly/")
taxi_df = spark.read.parquet("Cleaned_Parquet/ taxi_clean/")
weather_df = spark.read.parquet("Cleaned_Parquet/weather_clean/")
print("Combined df schema")
combined_df.printSchema()
print("City hourly schema")
city_hourly.printSchema()
print("Zone hourly schema")
zone_hourly.printSchema()
print("Weather schema")
weather_df.printSchema()



Load geojson data

In [ ]:
zones_geo = gpd.read_file("taxi_zones.geojson")  # contains Zone + LocationID
print(zones_geo.columns)


## Question 1: How Does Hourly Taxi Demand Vary Throughout the Day? ##

In [ ]:
# Q1: How does hourly taxi demand vary throughout the day?
demand_by_hour = city_hourly.groupBy('hour').agg(avg('trip_count').alias('avg_trips')).orderBy('hour')
demand_by_hour.show(24)

# Export
demand_by_hour.coalesce(1).write.mode('overwrite').csv('outputs/demand_by_hour.csv', header=True)

In [ ]:
import plotly.express as px

# Convert to pandas (24 rows — safe)
df_hourly = demand_by_hour.toPandas()

fig = px.line(
    df_hourly,
    x="hour",
    y="avg_trips",
    title="Average NYC Taxi Demand by Hour of Day",
    markers=True,
    labels={"hour": "Hour of Day (0–23)", "avg_trips": "Avg Trips per Hour"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis=dict(dtick=1),
    title_font_size=20,
    yaxis_title="Avg Trips",
    xaxis_title="Hour"
)

fig.show()


In [ ]:
import plotly.express as px
from pyspark.sql import functions as F
zone_demand = combined_df.groupBy("PULocationID").agg(
    F.count("*").alias("total_trips"),
    F.avg("fare_amount").alias("avg_fare"),
    F.avg("duration_min").alias("avg_trip_duration")
)
zone_pd = zone_demand.toPandas()
zones_geo["location_id"] = zones_geo["location_id"].astype(int)
zones_merged = zones_geo.merge(zone_pd, left_on="location_id", right_on="PULocationID", how="left")
zones_merged["total_trips"] = zones_merged["total_trips"].fillna(0)
fig = px.choropleth_mapbox(
    zones_merged,
    geojson=zones_merged.geometry.__geo_interface__,
    locations=zones_merged.index,
    color="total_trips",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.7,
    color_continuous_scale="YlOrRd",
    title="NYC Taxi Demand by Pickup Zone",
)

fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Extract date only
daily_zone = combined_df.withColumn("pickup_date", F.to_date("pickup_hour"))

# Aggregate trips per day per zone
daily_counts = (
    daily_zone.groupBy("PULocationID", "pickup_date")
    .agg(F.count("*").alias("trips"))
)

# Now average those daily trip totals for each zone
zone_avg_daily = (
    daily_counts.groupBy("PULocationID")
    .agg(F.avg("trips").alias("avg_trips_per_day"))
)
zone_avg_pd = zone_avg_daily.toPandas()
# Convert ID type to match
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_merged_avg = zones_geo.merge(
    zone_avg_pd, left_on="location_id", right_on="PULocationID", how="left"
)

zones_merged_avg["avg_trips_per_day"] = zones_merged_avg["avg_trips_per_day"].fillna(0)

import plotly.express as px

fig = px.choropleth_mapbox(
    zones_merged_avg,
    geojson=zones_merged_avg.geometry.__geo_interface__,
    locations=zones_merged_avg.index,
    color="avg_trips_per_day",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.8,
    color_continuous_scale="YlOrRd",
    title="Average Daily Taxi Demand by Pickup Zone",
)

fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()




Demand peaks around 6 pm , before decreasing throughout the night, eventually reaching it's minimum between 3 and 5 am. The number of trips rises steeply between 5 and 9 am, and then continues to grow at a slower steady pace throughout the late morning and afternoon before reaching peak hours in the evening

## Question 2: Which NYC zones have the highest pickup activity? ##

In [ ]:
# Q2: Which NYC zones have the highest pickup activity?
zone_totals = combined_df.groupBy('PULocationID').agg(count('*').alias('total_trips')).orderBy(col('total_trips').desc())
zone_totals.show(20)

#zone_totals.coalesce(1).write.mode('overwrite').csv('outputs/zone_totals.csv', header=True)
zone_totals.write.mode('overwrite').parquet('outputs/zone_totals.parquet')



In [ ]:
from pyspark.sql.functions import col, count

# Compute pickups per zone
zone_demand = (
    taxi_df
    .groupBy("PULocationID")
    .agg(count("*").alias("total_pickups"))
    .orderBy(col("total_pickups").desc())
)

zone_pd = zone_demand.toPandas()

import plotly.express as px
fig = px.bar(
    zone_pd.head(20),  # top 20 busiest zones
    x='PULocationID',
    y='total_pickups',
    title='Top 20 NYC Pickup Zones by Activity',
    text_auto=True,
    template='plotly_dark'
)
fig.update_layout(xaxis_title='Pickup Zone (ID)', yaxis_title='Total Pickups')
fig.show()


### Analysis ###


The results show that the highest taxi pickup activity is heavily concentrated in Manhattan, particularly in the Upper East Side and Midtown regions. The busiest zones include Upper East Side North (PULocationID 237), Midtown Center (161), and Upper East Side South (236), each recording around four million or more total pickups. Other major hotspots such as Midtown North, Murray Hill–Kips Bay, Times Square, and several Upper West Side zones also rank within the top 20. These areas are characterized by dense residential neighborhoods, major business districts, high tourism, and large transit hubs, all of which contribute to consistently strong taxi demand. Overall, the results highlight that Manhattan dominates taxi pickup activity across the city.

## Question 3: How does rain affect taxi demand / number of pickups ? ##

Does rain increase or decrease taxi demand on a per-hour basis? (Did raining hours have more pickups than non-rain hours?)

In [ ]:
#using 2 groups: raining vs not raining 

from pyspark.sql import functions as F

hourly_counts = (
    combined_df
    .groupBy("pickup_hour", "hour", "is_rainy")
    .agg(F.count("*").alias("trip_count"))
)
avg_by_rain = (
    hourly_counts
    .groupBy("is_rainy")
    .agg(F.avg("trip_count").alias("avg_trips_per_hour"))
    .orderBy("is_rainy")  
)
avg_by_rain.show()

#4 groups based on precipitation amt 
combined_df_prec_levels = (
    combined_df
    .withColumn(
        "precip_level",
        F.when(F.col("rain_mm") == 0, "None")
         .when((F.col("rain_mm") > 0) & (F.col("rain_mm") <= 1), "Light")
         .when((F.col("rain_mm") > 1) & (F.col("rain_mm") <= 5), "Moderate")
         .otherwise("Heavy")
    )
)
hourly_counts_prec_levels = (
    combined_df_prec_levels
    .groupBy("pickup_hour", "hour", "precip_level")
    .agg(F.count("*").alias("trip_count"))
)
avg_by_precip = (
    hourly_counts_prec_levels
    .groupBy("precip_level")
    .agg(F.avg("trip_count").alias("avg_trips_per_hour"))
    .orderBy("precip_level")
)

avg_by_precip.show()


Does rain change the distribution of demand across time?

In [ ]:
# 2 groups 
hourly_distribution = (
    hourly_counts
    .groupBy("hour", "is_rainy")
    .agg(F.avg("trip_count").alias("avg_trips"))
    .orderBy("hour", "is_rainy")
)

hourly_pivot = (
    hourly_distribution
    .groupBy("hour")
    .pivot("is_rainy", [0, 1])
    .agg(F.first("avg_trips"))
    .withColumnRenamed("0", "avg_trips_clear")
    .withColumnRenamed("1", "avg_trips_rain")
    .orderBy("hour")
)
hourly_pivot.show(24)

##Plot results:

import pandas
import plotly.express as px
hourly_pd = hourly_pivot.toPandas()
fig = px.line(
    hourly_pd,
    x="hour",
    y=["avg_trips_clear", "avg_trips_rain"],
    title="Hourly Taxi Demand: Rain vs Clear",
    labels={"value": "Average Trips", "variable": "Weather"},
    markers=True,
    template="plotly_dark"
)
fig.update_layout(
    xaxis=dict(dtick=1),
    legend_title_text="Condition"
)
fig.show()

In [ ]:
## 4 groups 
import plotly.express as px

hourly_distribution_prec_levels = (
    hourly_counts_prec_levels
    .groupBy("hour", "precip_level")
    .agg(F.avg("trip_count").alias("avg_trips"))
    .orderBy("hour", "precip_level")
)
hourly_pivot_prec_levels = (
    hourly_distribution_prec_levels
    .groupBy("hour")
    .pivot("precip_level", ["None", "Light", "Moderate", "Heavy"])
    .agg(F.first("avg_trips"))
    .orderBy("hour")
)
hourly_pivot_prec_levels.show(24)

prec_levels_pd = hourly_pivot_prec_levels.toPandas()
prec_levels_pd = prec_levels_pd.rename(columns={
    "None": "No Rain",
    "Light": "Light Rain",
    "Moderate": "Moderate Rain",
    "Heavy": "Heavy Rain"
})
fig = px.line(
    prec_levels_pd,
    x="hour",
    y=["No Rain", "Light Rain", "Moderate Rain", "Heavy Rain"],
    title="Hourly Taxi Demand by Precipitation Level",
    labels={"value": "Average Trips", "variable": "Precipitation"},
    markers=True,
    template="plotly_dark"
)

fig.update_layout(xaxis=dict(dtick=1))
fig.show()

### Written Analysis ##

These results suggest that in general, rain does not significantly affect taxi demand, but that more severe levels of rain decrease demand. Lighter to moderate amounts of precipitation show little affect on total trips , as well as the distribution of demand throughout the day, meaning that the average demand for a given hour of the day is not impacted significantly by light to moderate rain. There does seem to be a slight decrease in demand for light and moderate rain compared to no rain, but it is no where near proportional to the drop in demand when there is heavy precipitation. 

We initially hypothesized that more rain might increase the number of taxi trips, because people would want to avoid walking in the rain. However, it seems that instead rain might make people less likely to be out and about, therefore decreasing the number of taxi rides needed. 

## Question 4: Do colder temperatures lead to higher taxi usage? ##

In [ ]:
from pyspark.sql import functions as F

combined_df_temp = (
    combined_df
    .withColumn(
        "temp_level",
        F.when(F.col("temperature_c") < 0, "Very Cold")
         .when(F.col("temperature_c") < 5, "Cold")
         .when(F.col("temperature_c") < 15, "Cool")
         .otherwise("Warm")
    )
)

hourly_counts_temp = (
    combined_df_temp
    .groupBy("pickup_hour", "hour", "temp_level")
    .agg(F.count("*").alias("trip_count"))
)

avg_by_temp = (
    hourly_counts_temp
    .groupBy("temp_level")
    .agg(F.avg("trip_count").alias("avg_trips_per_hour"))
    .orderBy("avg_trips_per_hour", ascending=False)
)
avg_by_temp.show()
hourly_distribution_temp = (
    hourly_counts_temp
    .groupBy("hour", "temp_level")
    .agg(F.avg("trip_count").alias("avg_trips"))
    .orderBy("hour", "temp_level")
)
hourly_distribution_temp_pivot = (
    hourly_distribution_temp
    .groupBy("hour")
    .pivot("temp_level", ["Very Cold", "Cold", "Cool", "Warm"])
    .avg("avg_trips")
    .orderBy("hour")
)
hourly_distribution_temp_pivot.show()

In [ ]:
import plotly.express as px

hourly_temp_pd = hourly_distribution_temp.toPandas()

fig = px.line(
    hourly_temp_pd,
    x="hour",
    y="avg_trips",
    color="temp_level",
    title="Taxi Demand by Hour Under Different Temperature Levels",
    markers=True,
    template="plotly_dark"
)
fig.update_layout(
    xaxis=dict(dtick=1),
    legend_title_text="Temperature Level"
)
fig.show()


### Written Analysis ###

Here we can very clearly see that temperature has a direct impact on taxi demand. Very cold temperatures have the highest taxi demand at all hours of the day, and contribute to an even larger peak during the typical overall peak hours. Cold temperatures have a similar but lesser effect, and cool temperatures hover close to warm temperatures or slightly higher. 

## Question 5: Are trip durations longer during bad weather? ##

In [ ]:
duration_df_1 = combined_df.withColumn(
    "duration_min",
    (F.unix_timestamp("tpep_dropoff_datetime") -
     F.unix_timestamp("tpep_pickup_datetime")) / 60
)
avg_duration_1 = (
    duration_df_1
    .groupBy("is_rainy")
    .agg(F.avg("duration_min").alias("avg_duration_min"))
)
avg_duration_1.show()

In [ ]:
duration_precip = combined_df.withColumn(
    "precip_level",
    F.when(F.col("precip_mm") == 0, "0. None")
     .when(F.col("precip_mm") < 1, "1. Light (<1mm)")
     .when(F.col("precip_mm") < 3, "2. Moderate (1–3mm)")
     .otherwise("3. Heavy (>3mm)")
)
avg_duration_precip = (
    duration_precip
    .groupBy("precip_level")
    .agg(F.avg("duration_min").alias("avg_duration_min"))
    .orderBy("precip_level")
)

avg_duration_precip.show()

duration_quantiles = (
    duration_precip
    .groupBy("precip_level")
    .agg(F.expr("percentile(duration_min, 0.5)").alias("median_duration"),
         F.expr("percentile(duration_min, 0.9)").alias("p90_duration"))
)
pd_precip = avg_duration_precip.toPandas()

import plotly.express as px
fig = px.bar(
    pd_precip,
    x="precip_level",
    y="avg_duration_min",
    title="Average Trip Duration by Precipitation Level",
    text_auto=".2f",
    template="plotly_dark"
)
fig.show()


In [ ]:
duration_temp = combined_df.withColumn(
    "temp_level",
    F.when(F.col("temperature_c") < 5, "Cold (<5°C)")
    .when(F.col("temperature_c") > 25, "Hot (>25°C)")
    .otherwise("Mild (5–25°C)")
)

avg_duration_temp = (
    duration_temp
    .groupBy("temp_level")
    .agg(F.avg("duration_min").alias("avg_duration_min"))
    .orderBy("temp_level")
)

avg_duration_temp.show()

temp_stats = (
    duration_temp
    .groupBy("temp_level")
    .agg(
        F.avg("duration_min").alias("avg_duration"),
        F.expr("percentile(duration_min, 0.5)").alias("median_duration")
    )
    .orderBy("temp_level")
)

pd_temp = temp_stats.toPandas()

fig = px.bar(
    pd_temp,
    x="temp_level",
    y=["avg_duration", "median_duration"],
    barmode="group",
    title="Trip Duration vs Temperature Category",
    labels={"value": "Duration (minutes)", "temp_level": "Temperature Level"},
    text_auto=".1f",
    template="plotly_dark"
)
fig.show()



### Analysis ###

Based on these findings, it appears that temperature and precipitation both have very little to no effect on trip duration. Any observable difference is more likely due to traffic patterns rather than customer habits.

## Question 6: Which zones experience the highest fare surges?  ##

In [ ]:
# Q6: Which zones experience the highest fare surges?
from pyspark.sql.functions import first

zone_fare_weather = combined_df.groupBy('PULocationID','is_rainy').agg(avg('fare_amount').alias('avg_fare'))
zone_fare_pivot = zone_fare_weather.groupBy('PULocationID').pivot('is_rainy', [0,1]).agg(first('avg_fare'))
# rename pivot columns if they exist
cols = zone_fare_pivot.columns
if '0' in cols:
    zone_fare_pivot = zone_fare_pivot.withColumnRenamed('0','fare_clear')
if '1' in cols:
    zone_fare_pivot = zone_fare_pivot.withColumnRenamed('1','fare_rainy')

zone_fare_pivot = zone_fare_pivot.withColumn('pct_increase', (col('fare_rainy') - col('fare_clear'))/col('fare_clear')*100)
zone_fare_pivot.orderBy(col('pct_increase').desc()).show(20)

#zone_fare_pivot.coalesce(1).write.mode('overwrite').csv('outputs/zone_fare_pivot.csv', header=True)

We compared average taxi fares from each pickup zone during clear conditions versus rainy conditions. Several zones showed significant fare increases, in some cases more than 40–60% higher when it was raining. Because NYC yellow taxis do not use surge pricing, these increases are most likely driven by travel delays caused by rain, such as heavier traffic congestion, reduced road speeds, and longer trip durations. Zones near major transit choke-points or dense commercial areas appear to be the most affected.

## Question 7: What is the correlation between precipitation and city-wide taxi demand, and between temperature and city-wide taxi demand? ##

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import functions as F

hourly = combined_df.groupBy("pickup_hour", "hour").agg(
    F.sum("trip_distance").alias("total_distance"),
    F.count("*").alias("trip_count"),
    F.avg("rain_mm").alias("rain_mm"),
    F.avg("temperature_c").alias("temperature_c")
)
corr_by_hour = (
    hourly.groupBy("hour")
    .agg(F.corr("rain_mm", "trip_count").alias("corr_rain_demand"))
    .orderBy("hour")
)
corr_by_hour.show(24)



In [ ]:
from pyspark.sql.functions import count, avg

hourly_weather = combined_df.groupBy(
    "year", "month", "day", "hour"
).agg(
    count("*").alias("trips"),
    avg("rain_mm").alias("rain_mm")
)
df_clean = hourly_weather.dropna(subset=["rain_mm", "trips"]).toPandas()
corr = df_clean['rain_mm'].corr(df_clean['trips'])
print(f"Correlation between rain and taxi demand: {corr:.3f}")
fig = px.scatter(
    df_clean,
    x='rain_mm',
    y='trips',
    trendline='ols',
    title=f"Hourly Rain vs Taxi Demand (Correlation = {corr:.2f})",
    template='plotly_dark'
)
fig.update_layout(
    xaxis_title="Rain (mm)",
    yaxis_title="Number of Trips"
)
fig.show()


In [ ]:
import plotly.express as px
import pyspark.sql.functions as F
""" 
rain_vs_demand = hourly.select("rain_mm", "trip_count")
rain_pd = rain_vs_demand.toPandas()
rain_pd = rain_pd.dropna(subset=['rain_mm', 'trip_count'])
corr = rain_pd['rain_mm'].corr(rain_pd['trip_count'])
print(f"Correlation between rain and taxi demand: {corr:.3f}")
fig = px.scatter(
    rain_pd,
    x='rain_mm',
    y='trip_count',
    trendline='ols',
    title=f"Hourly Rain vs Taxi Demand (Correlation = {corr:.3f})",
    template='plotly_dark'
)
fig.update_layout(
    xaxis_title="Rainfall (mm per hour)",
    yaxis_title="Taxi Trips per Hour"
)
fig.show() """


In [ ]:
from pyspark.sql import functions as F

corr_temp_by_hour = (
    hourly.groupBy("hour")
    .agg(F.corr("temperature_c", "trip_count").alias("corr_temp_demand"))
    .orderBy("hour")
)

corr_temp_by_hour.show(24)

In [ ]:
""" temp_pd = hourly.select("temperature_c", "trip_count").toPandas()
corr_temp = temp_pd['temperature_c'].corr(temp_pd['trip_count'])
print(f"Correlation between temperature and taxi demand: {corr_temp:.3f}")
import plotly.express as px

fig = px.scatter(
    temp_pd,
    x='temperature_c',
    y='trip_count',
    trendline='ols',   # remove this line if you still get statsmodels error
    title=f"Hourly Temperature vs Taxi Demand (Correlation = {corr_temp:.2f})",
    template='plotly_dark'
)

fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Taxi Trips per Hour"
)

fig.show()
 """

In [ ]:
from pyspark.sql.functions import count, avg
import plotly.express as px

# Create the same structure but for temperature
hourly_temp = combined_df.groupBy(
    "year", "month", "day", "hour"
).agg(
    count("*").alias("trips"),
    avg("temperature_c").alias("temperature_c")
)

# Convert to Pandas + drop missing
df_clean = hourly_temp.dropna(subset=["temperature_c", "trips"]).toPandas()

# Compute correlation
corr = df_clean['temperature_c'].corr(df_clean['trips'])
print(f"Correlation between temperature and taxi demand: {corr:.3f}")

# Plot
fig = px.scatter(
    df_clean,
    x='temperature_c',
    y='trips',
    trendline='ols',
    title=f"Hourly Temperature vs Taxi Demand (Correlation = {corr:.3f})",
    template='plotly_dark'
)
fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Number of Trips"
)
fig.show()

Analysis of hourly correlations shows that rain has only a weak and inconsistent relationship with taxi demand, with values hovering near zero across all hours, suggesting that precipitation alone does not reliably increase or decrease citywide ridership. In contrast, colder temperatures show a more consistent negative correlation with demand, indicating that taxi use tends to rise as temperatures drop, likely because walking and other outdoor travel becomes less comfortable. Overall, temperature appears to be a more reliable driver of demand than rain, while precipitation’s impact is smaller and more context-dependent.

## Question 8: What are the best predictors of hourly taxi demand ?  ##

In [ ]:
# Q8: Best predictors of hourly taxi demand using RandomForest feature importance
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# Prepare hourly weather averaged features
hourly_weather_avg = weather_df.groupBy('weather_time') \
    .agg(avg('temperature_c').alias('avg_temp'), avg('rain_mm').alias('avg_rain'), avg('windspeed_kmh').alias('avg_wind')) \
    .withColumnRenamed('weather_time', 'pickup_hour')

ml_df = city_hourly.join(hourly_weather_avg, on='pickup_hour', how='left').na.fill(0)
ml_df = ml_df.withColumn('hour', hour(col('pickup_hour'))).withColumn('day_of_week', dayofweek(col('pickup_hour')))

assembler = VectorAssembler(inputCols=['hour','day_of_week','avg_temp','avg_rain','avg_wind'], outputCol='features')
rf = RandomForestRegressor(featuresCol='features', labelCol='trip_count', numTrees=50)
pipeline = Pipeline(stages=[assembler, rf])

train_df, test_df = ml_df.randomSplit([0.8,0.2], seed=42)
model = pipeline.fit(train_df)
preds = model.transform(test_df)

evaluator = RegressionEvaluator(labelCol='trip_count', predictionCol='prediction', metricName='rmse')
print('RMSE:', evaluator.evaluate(preds))

rf_model = model.stages[-1]
print('Feature importances:', rf_model.featureImportances)

The model’s feature importance scores show that time-based variables are by far the strongest predictors, with hour of the day accounting for roughly 83% of the predictive power and day of week contributing another 6%. Weather-related features played a much smaller role: temperature showed a modest effect (~8%), while rain and wind contributed very little. This indicates that demand in New York City is primarily driven by predictable daily and weekly travel patterns rather than short-term weather changes, with weather factors influencing demand only marginally compared to people’s regular commuting schedules and activity cycles.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

hourly_weather = combined_df.groupBy(
    "year", "month", "day", "hour"
).agg(
    count("*").alias("trip_count"),
    avg("rain_mm").alias("rain_mm")
)

hourly_weather = hourly_weather.withColumn(
    "pickup_hour",
    to_timestamp(concat_ws(" ", concat_ws("-", "year", "month", "day"), concat_ws(":", "hour")))
)
hourly_weather = hourly_weather.dropna(subset=["rain_mm", "trip_count"]).toPandas()
# --- Feature Engineering ---
df = hourly_weather
df['hour'] = df['pickup_hour'].dt.hour
df['weekday'] = df['pickup_hour'].dt.dayofweek  # Monday=0, Sunday=6
df['month'] = df['pickup_hour'].dt.month
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)

# Clean missing values
df = df.dropna(subset=['trip_count'])
df = df.fillna(0)

# --- Define features and target ---
feature_cols = ['hour', 'weekday', 'month', 'is_weekend', 'rain_mm']
X = df[feature_cols]
y = df['trip_count']

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Train Model ---
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# --- Evaluate Performance ---
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"Model R² score: {r2:.3f}")

# --- Feature Importance ---
importances = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=True)

plt.figure(figsize=(8, 5))
importances.plot(kind='barh', color='skyblue')
plt.title("Feature Importance: Predictors of Hourly Taxi Demand")
plt.xlabel("Importance Score")
plt.show()


## Question 9: Forecast next-day demand using lag features + RandomForest ##

In [ ]:
# Q9: Forecast next-day demand using lag features + RandomForest
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

city_hourly = city_hourly.orderBy('pickup_hour')
w = Window.orderBy('pickup_hour')

lagged = city_hourly.withColumn('lag_1', lag('trip_count',1).over(w)).withColumn('lag_24', lag('trip_count',24).over(w)).na.fill(0)
lagged = lagged.withColumn('hour', hour(col('pickup_hour'))).withColumn('day_of_week', dayofweek(col('pickup_hour')))

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor

assembler = VectorAssembler(inputCols=['lag_1','lag_24','hour','day_of_week'], outputCol='features')
ml_lag = assembler.transform(lagged).select('features','trip_count')

train, test = ml_lag.randomSplit([0.8,0.2], seed=42)
rf2 = RandomForestRegressor(featuresCol='features', labelCol='trip_count', numTrees=50)
model2 = rf2.fit(train)
preds2 = model2.transform(test)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol='trip_count', predictionCol='prediction', metricName='rmse')
print('Forecast RMSE:', evaluator.evaluate(preds2))


To explore whether we could forecast taxi demand one day ahead, we created a predictive model using lag features, meaning information from previous hours. We added two key inputs: the demand from one hour earlier (lag_1) and the demand from the same hour on the previous day (lag_24), along with the hour of day and day of week. Using a Random Forest regression model, we trained the model to predict future hourly demand based on these patterns. The model achieved an RMSE of approximately 1,010, which is significantly lower than the baseline RMSE from earlier models without lagged features. This suggests that the strongest predictors of tomorrow’s demand are simply yesterday’s demand at the same time and the general hourly routine of the city, meaning New York’s taxi patterns are highly repetitive and stable across days. As a result, short-term historical demand proves to be more useful for forecasting than standalone weather or timing variables.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt

hourly_weather = combined_df.groupBy(
    "year", "month", "day", "hour"
).agg(
    count("*").alias("trip_count"),
    avg("rain_mm").alias("rain_mm")
)

hourly_weather = hourly_weather.withColumn(
    "pickup_hour",
    to_timestamp(concat_ws(" ", concat_ws("-", "year", "month", "day"), concat_ws(":", "hour")))
)
hourly_weather = hourly_weather.dropna(subset=["rain_mm", "trip_count"]).toPandas()
df = hourly_weather
# --- Feature Engineering ---
df['hour'] = df['pickup_hour'].dt.hour
df['weekday'] = df['pickup_hour'].dt.dayofweek
df['month'] = df['pickup_hour'].dt.month
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)

# --- Handle missing data ---
df = df.dropna(subset=['trip_count'])
df = df.fillna(0)

# --- If you have these extra columns from weather data, include them ---
# (If not, the model will still run with rain only)
possible_weather_cols = ['rain_mm', 'temperature_2m (°C)', 'windspeed_10m (km/h)', 'humidity (%)']
existing_weather_cols = [c for c in possible_weather_cols if c in df.columns]

feature_cols = ['hour', 'weekday', 'month', 'is_weekend'] + existing_weather_cols

# --- Define Features and Target ---
X = df[feature_cols]
y = df['trip_count']

# --- Simulate "next-day" forecasting ---
# Sort by time to simulate temporal prediction
df_sorted = df.sort_values('pickup_hour')
split_point = int(len(df_sorted) * 0.8)
X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

# --- Train Model ---
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

# --- Evaluate Forecasting Accuracy ---
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Forecasting R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.2f} trips")

# --- Plot Actual vs Predicted Demand ---
plt.figure(figsize=(10,5))
plt.plot(y_test.values[:200], label='Actual', color='lightgreen')
plt.plot(y_pred[:200], label='Predicted', color='orange')
plt.title("Next-Day Taxi Demand Forecast (Sample 200 Hours)")
plt.xlabel("Hour Index")
plt.ylabel("Trip Count")
plt.legend()
plt.show()

# --- Feature Importance ---
importances = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=True)
plt.figure(figsize=(8,5))
importances.plot(kind='barh', color='skyblue')
plt.title("Feature Importance: Taxi Demand Forecasting")
plt.xlabel("Importance Score")
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
df['trip_count_next_day'] = df['trip_count'].shift(-24)

# Drop rows where target is NaN (last 24 rows)
df = df.dropna(subset=['trip_count_next_day'])

# Features and target
feature_cols = ['hour', 'weekday', 'month', 'is_weekend', 'rain_mm']
X = df[feature_cols]
y = df['trip_count_next_day']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"R² score: {r2:.3f}, RMSE: {rmse:.2f}")

# Feature importance
importances = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=True)
plt.figure(figsize=(8,5))
importances.plot(kind='barh', color='skyblue')
plt.title("Feature Importance: Predictors of Next-Day Taxi Demand")
plt.xlabel("Importance Score")
plt.show()

In [ ]:
import seaborn as sns
df['rain_level'] = pd.cut(df['rain_mm'], bins=[-0.1, 2, 10, 100], labels=['Light', 'Moderate', 'Heavy'])

# Compute average demand per hour and rain level
heatmap_data = df.groupby(['hour', 'rain_level'])['trip_count'].mean().unstack()

# Plot heatmap
plt.figure(figsize=(12,6))
sns.heatmap(heatmap_data, annot=True, fmt=".0f", cmap='YlOrRd', cbar_kws={'label': 'Expected Trips'})
plt.title("Next-Day Taxi Demand by Hour and Rain Level")
plt.xlabel("Rain Level")
plt.ylabel("Hour of Day")
plt.show()

In [ ]:
heatmap_data = df.groupby(['hour', 'rain_level', 'is_weekend'])['trip_count'].mean().reset_index()

# Separate weekend and weekday
weekday_data = heatmap_data[heatmap_data['is_weekend'] == 0].pivot(index='hour', columns='rain_level', values='trip_count')
weekend_data = heatmap_data[heatmap_data['is_weekend'] == 1].pivot(index='hour', columns='rain_level', values='trip_count')

# --- Plot Heatmaps ---
fig, axes = plt.subplots(1, 2, figsize=(18,6), sharey=True)

sns.heatmap(weekday_data, annot=True, fmt=".0f", cmap='YlOrRd', ax=axes[0], cbar_kws={'label': 'Expected Trips'})
axes[0].set_title("Weekday Taxi Demand by Hour & Rain Level")
axes[0].set_xlabel("Rain Level")
axes[0].set_ylabel("Hour of Day")

sns.heatmap(weekend_data, annot=True, fmt=".0f", cmap='YlOrRd', ax=axes[1], cbar_kws={'label': 'Expected Trips'})
axes[1].set_title("Weekend Taxi Demand by Hour & Rain Level")
axes[1].set_xlabel("Rain Level")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

## Q10 : Recommendations — compute zones with largest uplift and export ##

### Rain uplift ##

In [ ]:
from pyspark.sql import functions as F

# average trips per hour by zone & rain
zone_hourly = combined_df.groupBy("PULocationID", "is_rainy") \
    .agg(F.count("*").alias("trips"), F.countDistinct("pickup_hour").alias("hours"))
zone_avg = zone_hourly.withColumn("avg_trips_per_hour", F.col("trips") / F.col("hours"))
# pivot to compare rain vs clear
zone_uplift = zone_avg.groupBy("PULocationID").pivot("is_rainy", [0,1]).agg(F.first("avg_trips_per_hour"))
zone_uplift = (
    zone_uplift
    .withColumnRenamed("0", "avg_clear")
    .withColumnRenamed("1", "avg_rain")
    .withColumn("pct_uplift", (F.col("avg_rain") - F.col("avg_clear")) / F.col("avg_clear") * 100)
    .orderBy("pct_uplift", ascending=False)
)
zone_uplift.show(20)

### Temperature Uplift (Cold vs Mild) ###

In [ ]:
from pyspark.sql import functions as F

combined_binned = combined_df.withColumn(
    "temp_bin",
    F.when(F.col("temperature_c") < 5, "Cold (<5°C)")
     .when(F.col("temperature_c") < 15, "Cool (5–15°C)")
     .when(F.col("temperature_c") < 25, "Mild (15–25°C)")
     .otherwise("Hot (>25°C)")
)
zone_temp = (
    combined_binned
    .groupBy("PULocationID", "temp_bin")
    .agg(
        F.count("*").alias("trips"),
        F.countDistinct("pickup_hour").alias("hours")
    )
)
zone_temp = zone_temp.withColumn(
    "avg_trips_per_hour",
    F.col("trips") / F.col("hours")
)
zone_temp_pivot = (
    zone_temp
    .groupBy("PULocationID")
    .pivot("temp_bin", ["Cold (<5°C)", "Cool (5–15°C)", "Mild (15–25°C)", "Hot (>25°C)"])
    .agg(F.first("avg_trips_per_hour"))
)

zone_temp_results = zone_temp_pivot.withColumn(
    "pct_uplift_cold_vs_mild",
    (F.col("Cold (<5°C)") - F.col("Mild (15–25°C)")) / F.col("Mild (15–25°C)") * 100
)
zone_temp_results.orderBy("pct_uplift_cold_vs_mild", ascending=False).show(20)


### Analysis ###

The uplift analysis highlights that weather influences taxi demand differently across the city’s pickup zones. A small number of zones show modest increases in demand during rain (up to roughly 28% uplift), suggesting these areas may rely more on taxis when conditions discourage walking or transit use. However, temperature appears to have a much stronger and more consistent impact: several zones show a 30–60% increase in demand during colder conditions compared to mild temperatures, indicating that cold weather is a more powerful driver of taxi usage than rain. These results suggest that demand shifts caused by weather are highly location-dependent, and that temperature-related effects are substantially more significant than precipitation when planning for fleet allocation or forecasting demand across NYC.

In [ ]:

# Lag feature: previous day's same hour demand
df = df.sort_values('pickup_hour')
df['prev_day_demand'] = df['trip_count'].shift(24)

# Drop rows without lag data
df = df.dropna(subset=['prev_day_demand'])

# Features & target
feature_cols = ['hour', 'weekday', 'month', 'is_weekend', 'rain_mm', 'prev_day_demand']
X = df[feature_cols]
y = df['trip_count']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)

# Evaluate
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Forecast R²: {r2:.3f}")
print(f"MAE: {mae:.2f} trips")
print(f"RMSE: {rmse:.2f} trips")


In [ ]:
feature_cols = ['hour', 'weekday', 'month', 'is_weekend', 'rain_mm', 'prev_day_demand']

# Scenarios for weather
rain_scenarios = {
    'no_rain': 0.0,
    'light_rain': 2.0,
    'heavy_rain': 10.0
}

recommendations = []

for scenario, rain_val in rain_scenarios.items():
    X_scenario = df.copy()
    X_scenario['rain_mm'] = rain_val
    X_scenario_features = X_scenario[feature_cols]
    
    # Predict demand
    y_pred = rf.predict(X_scenario_features)
    
    X_scenario[f'predicted_demand_{scenario}'] = y_pred
    recommendations.append(X_scenario[['pickup_hour', f'predicted_demand_{scenario}']])

# Merge scenarios
rec_table = recommendations[0]
for r in recommendations[1:]:
    rec_table = rec_table.merge(r, on='pickup_hour')

# Fleet allocation suggestion: add 10% buffer for heavy rain
rec_table['fleet_suggestion'] = rec_table['predicted_demand_heavy_rain'] * 1.1
rec_table['fleet_suggestion'] = rec_table['fleet_suggestion'].apply(np.ceil)  # round up

# Show sample
print(rec_table.head())

# Save to CSV
rec_table.to_csv("taxi_fleet_recommendations.csv", index=False)


### Additional Visualizations

### Heatmaps ###

Comparison: Avg Pickups per hour - Non-Rainy hour vs Rainy hour

In [ ]:
from pyspark.sql import functions as F

from pyspark.sql.functions import when, col

combined_df_2= combined_df.withColumn(
    "is_rainy_heavy",
    when(col("rain_mm") > 1.0, 1).otherwise(0)
)
combined_df_2.printSchema()

# Calculate avg trips per hour by zone and rain condition
rain_zone_hourly = combined_df_2.groupBy("PULocationID", "is_rainy") \
    .agg(F.count("*").alias("total_trips"),
         F.countDistinct("pickup_hour").alias("num_hours"))

# Calculate avg trips per hour
rain_zone_hourly = rain_zone_hourly.withColumn(
    "avg_trips_per_hour",
    F.when(F.col("num_hours") > 0, F.col("total_trips") / F.col("num_hours")).otherwise(0)
)

# Pivot so we have columns: trips_clear / trips_rainy
rain_zone_pivot = rain_zone_hourly.groupBy("PULocationID") \
    .pivot("is_rainy", [0,1]) \
    .agg(F.first("avg_trips_per_hour"))

# Rename pivoted columns
rain_zone_pivot = (
    rain_zone_pivot
    .withColumnRenamed("0", "avg_trips_clear")
    .withColumnRenamed("1", "avg_trips_rain")
)

# Fill missing with zero (zones with no rain entries)
rain_zone_pd = rain_zone_pivot.toPandas()
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_rain_merged = zones_geo.merge(
    rain_zone_pd, left_on="location_id", right_on="PULocationID", how="left"
)

zones_rain_merged["avg_trips_clear"] = zones_rain_merged["avg_trips_clear"].fillna(0)
zones_rain_merged["avg_trips_rain"] = zones_rain_merged["avg_trips_rain"].fillna(0)
import numpy as np

zones_rain_merged["log_avg_clear"] = np.log1p(zones_rain_merged["avg_trips_clear"])



fig = px.choropleth_mapbox(
    zones_rain_merged,
    geojson=zones_rain_merged.geometry.__geo_interface__,
    locations=zones_rain_merged.index,
    color="avg_trips_clear",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Average Taxi Pickups per Hour — Clear Weather"
)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig2 = px.choropleth_mapbox(
    zones_rain_merged,
    geojson=zones_rain_merged.geometry.__geo_interface__,
    locations=zones_rain_merged.index,
    color="avg_trips_rain",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Average Taxi Pickups per Hour — Rainy Weather"
)
fig2.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig2.show()



In [ ]:
import numpy as np

zones_rain_merged["log_avg_clear"] = np.log1p(zones_rain_merged["avg_trips_clear"])

fig = px.choropleth_mapbox(
    zones_rain_merged,
    geojson=zones_rain_merged.geometry.__geo_interface__,
    locations=zones_rain_merged.index,
    color="log_avg_clear",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Viridis",  
    title="Average Taxi Pickups (Clear Weather) — Log Scaled"
)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()

**change below cell to use average not toal trips

In [ ]:
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px

# Create new rainy classification (>= 1mm precip)
combined_df = combined_df.withColumn(
    "is_rainy",
    F.when(F.col("rain_mm") >= 1, 1).otherwise(0)
)

# Compute avg trips per zone under clear vs rainy hours
zone_weather = combined_df.groupBy("PULocationID", "is_rainy").agg(
    F.count("*").alias("total_trips"),
    F.countDistinct("pickup_hour").alias("num_hours")
)

# Calculate avg trips per hour
zone_weather = zone_weather.withColumn(
    "avg_trips_per_hour",
    F.when(F.col("num_hours") > 0, F.col("total_trips") / F.col("num_hours")).otherwise(0)
)

zone_weather_pivot = (
    zone_weather.groupBy("PULocationID")
    .pivot("is_rainy", [0, 1])
    .agg(F.first("avg_trips_per_hour"))
)

zone_weather_pivot = (
    zone_weather_pivot.withColumnRenamed("0", "avg_trips_clear")
                      .withColumnRenamed("1", "avg_trips_rain")
                      .fillna(0)
)

# Convert to pandas
zone_pd = zone_weather_pivot.toPandas()

# Log-scale for better color contrast on map
zone_pd["log_clear"] = np.log1p(zone_pd["avg_trips_clear"])
zone_pd["log_rain"]  = np.log1p(zone_pd["avg_trips_rain"])

#Load NYC Taxi Zones GeoJSON & Merge
zones_geo = gpd.read_file("taxi_zones.geojson")
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_merged = zones_geo.merge(zone_pd, left_on="location_id", right_on="PULocationID", how="left")
zones_merged = zones_merged.fillna(0)

# Also create zones_rain_merged for later use (same data)
zones_rain_merged = zones_merged.copy()

# avg pickups clear weather
fig_clear = px.choropleth_mapbox(
    zones_merged,
    geojson=zones_merged.geometry.__geo_interface__,
    locations=zones_merged.index,
    color="log_clear",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Average Taxi Pickups — Clear Weather (Log Scaled)"
)
fig_clear.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_clear.show()

#avg pickups rainy weather
fig_rain = px.choropleth_mapbox(
    zones_merged,
    geojson=zones_merged.geometry.__geo_interface__,
    locations=zones_merged.index,
    color="log_rain",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Average Taxi Pickups — Rainy Weather (Log Scaled)"
)
fig_rain.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_rain.show()

zones_rain_merged["pct_uplift_rain"] = np.where(
    zones_rain_merged["avg_trips_clear"] > 0,
    ((zones_rain_merged["avg_trips_rain"] - zones_rain_merged["avg_trips_clear"]) /
     zones_rain_merged["avg_trips_clear"]) * 100,
    0
)

# Cap extreme values for better visualization
zones_rain_merged["pct_uplift_rain"] = zones_rain_merged["pct_uplift_rain"].clip(lower=-100, upper=100)

zones_rain_merged["log_avg_clear"] = np.log1p(zones_rain_merged["avg_trips_clear"])

fig_clear = px.choropleth_mapbox(
    zones_rain_merged,
    geojson=zones_rain_merged.geometry.__geo_interface__,
    locations=zones_rain_merged.index,
    color="log_avg_clear",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Viridis",
    title="Average Taxi Pickups (Clear Weather) — Log Scaled"
)
fig_clear.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_clear.show()

#percent uplift
fig_uplift = px.choropleth_mapbox(
    zones_rain_merged,
    geojson=zones_rain_merged.geometry.__geo_interface__,
    locations=zones_rain_merged.index,
    color="pct_uplift_rain",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.8,
    opacity=0.85,
    color_continuous_scale="Turbo",  # green=min, red=max
    title="Percentage Change in Taxi Demand — Rain vs Clear (%)"
)
fig_uplift.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_uplift.show()



In [ ]:
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

#Temperature bucket classification
combined_df_3 = combined_df.withColumn(
    "temp_bucket",
    F.when(F.col("temperature_c") < 5, "Cold (<5°C)")
     .when((F.col("temperature_c") >= 15) & (F.col("temperature_c") <= 25), "Mild (15–25°C)")
     .otherwise("Other")
)

# Compute avg pickups per zone & temperature category
zone_temp = combined_df_3.groupBy("PULocationID", "temp_bucket").agg(
    F.count("*").alias("total_trips"),
    F.countDistinct("pickup_hour").alias("num_hours")
)

# Calculate avg trips per hour
zone_temp = zone_temp.withColumn(
    "avg_trips_per_hour",
    F.when(F.col("num_hours") > 0, F.col("total_trips") / F.col("num_hours")).otherwise(0)
)

# Pivot so Cold and Mild become columns
zone_temp_pivot = (
    zone_temp.groupBy("PULocationID")
             .pivot("temp_bucket", ["Cold (<5°C)", "Mild (15–25°C)"])
             .agg(F.first("avg_trips_per_hour"))
             .fillna(0)
)

zone_temp_pd = zone_temp_pivot.toPandas()

# Compute uplift: (Cold - Mild)/Mild * 100
zone_temp_pd["pct_uplift_cold_vs_mild"] = np.where(
    zone_temp_pd["Mild (15–25°C)"] > 0,
    (zone_temp_pd["Cold (<5°C)"] - zone_temp_pd["Mild (15–25°C)"]) / zone_temp_pd["Mild (15–25°C)"] * 100,
    0
)

# Apply log scaling to demand (for heatmap clarity)
zone_temp_pd["log_cold"] = np.log1p(zone_temp_pd["Cold (<5°C)"])
zone_temp_pd["log_mild"] = np.log1p(zone_temp_pd["Mild (15–25°C)"])

# Load GeoJSON and merge
zones_geo = gpd.read_file("taxi_zones.geojson")
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_temp_merged = zones_geo.merge(
    zone_temp_pd, left_on="location_id", right_on="PULocationID", how="left"
).fillna(0)

# Cold Heatmap
fig_cold = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="log_cold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Heatmap: Average Taxi Pickups in COLD Weather (<5°C)"
)
fig_cold.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_cold.show()

# Mild heatmap
fig_mild = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="log_mild",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Heatmap: Average Taxi Pickups in MILD Weather (15–25°C)"
)
fig_mild.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_mild.show()

fig_uplift = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="pct_uplift_cold_vs_mild",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale="Turbo",
    title="Heatmap: Percentage Uplift in Demand — COLD vs MILD Weather (%)"
)
fig_uplift.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_uplift.show()


In [ ]:
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import col, when
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Create temp bucket column ("Cold (<5°C)" or "Not Cold")
combined_df_4 = combined_df.withColumn(
    "temp_bucket",
    when(col("temperature_c") < 5, "Cold (<5°C)").otherwise("Not Cold")
)

zone_temp = combined_df_4.groupBy("PULocationID", "temp_bucket").agg(
    F.count("*").alias("total_trips"),
    F.countDistinct("pickup_hour").alias("num_hours")
)

# Calculate avg trips per hour
zone_temp = zone_temp.withColumn(
    "avg_trips_per_hour",
    F.when(F.col("num_hours") > 0, F.col("total_trips") / F.col("num_hours")).otherwise(0)
)

zone_temp_pivot = (
    zone_temp.groupBy("PULocationID")
             .pivot("temp_bucket", ["Cold (<5°C)", "Not Cold"])
             .agg(F.first("avg_trips_per_hour"))
             .fillna(0)
)

# Convert to pandas
zone_temp_pd = zone_temp_pivot.toPandas()

zone_temp_pd["pct_uplift_cold_vs_notcold"] = np.where(
    zone_temp_pd["Not Cold"] > 0,
    (zone_temp_pd["Cold (<5°C)"] - zone_temp_pd["Not Cold"]) / zone_temp_pd["Not Cold"] * 100,
    0
)

# Log scale for clarity
zone_temp_pd["log_cold"] = np.log1p(zone_temp_pd["Cold (<5°C)"])
zone_temp_pd["log_notcold"] = np.log1p(zone_temp_pd["Not Cold"])

zones_geo = gpd.read_file("taxi_zones.geojson")
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_temp_merged = zones_geo.merge(
    zone_temp_pd, left_on="location_id", right_on="PULocationID", how="left"
).fillna(0)
fig_cold = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="log_cold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.90,
    color_continuous_scale="Turbo",
    title="Heatmap: Average Taxi Pickups in COLD Weather (< 5°C)"
)
fig_cold.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_cold.show()

fig_notcold = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="log_notcold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.90,
    color_continuous_scale="Turbo",
    title="Heatmap: Average Taxi Pickups in NOT COLD Weather (≥ 5°C)"
)
fig_notcold.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_notcold.show()

fig_uplift = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="pct_uplift_cold_vs_notcold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.90,
    color_continuous_scale="Turbo",
    title="Heatmap: % Uplift in Demand — Cold vs Not Cold (%)"
)
fig_uplift.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_uplift.show()


In [ ]:


from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Define PM peak hours
pm_peak_hours = [17, 18, 19]  # 5–7 PM

# Classify temperature buckets & filter to PM peak only
combined_df_peak = (
    combined_df
    .withColumn("temp_bucket",
        F.when(F.col("temperature_c") < 5, "Cold (<5°C)")
         .otherwise("Not Cold")
    )
    .withColumn("hour", F.hour("pickup_hour"))
    .filter(F.col("hour").isin(pm_peak_hours))
)

# Compute avg pickups by zone & temperature category (PM peak only)
zone_temp_peak = combined_df_peak.groupBy("PULocationID", "temp_bucket").agg(
    F.count("*").alias("total_trips"),
    F.countDistinct("pickup_hour").alias("num_hours")
)

# Calculate avg trips per hour
zone_temp_peak = zone_temp_peak.withColumn(
    "avg_trips",
    F.when(F.col("num_hours") > 0, F.col("total_trips") / F.col("num_hours")).otherwise(0)
)

# Pivot so Cold / Not Cold are columns
zone_temp_peak_pivot = (
    zone_temp_peak.groupBy("PULocationID")
                  .pivot("temp_bucket", ["Cold (<5°C)", "Not Cold"])
                  .agg(F.first("avg_trips"))
                  .fillna(0)
)

zone_temp_peak_pd = zone_temp_peak_pivot.toPandas()

# Percentage uplift (Cold vs Not Cold)
zone_temp_peak_pd["pct_uplift_cold_vs_notcold"] = np.where(
    zone_temp_peak_pd["Not Cold"] > 0,
    (zone_temp_peak_pd["Cold (<5°C)"] - zone_temp_peak_pd["Not Cold"]) / zone_temp_peak_pd["Not Cold"] * 100,
    0
)

# Log scaling for clarity
zone_temp_peak_pd["log_cold"] = np.log1p(zone_temp_peak_pd["Cold (<5°C)"])
zone_temp_peak_pd["log_notcold"] = np.log1p(zone_temp_peak_pd["Not Cold"])

# 6️⃣ Load GeoJSON & merge
zones_geo = gpd.read_file("taxi_zones.geojson")
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_temp_peak_merged = zones_geo.merge(
    zone_temp_peak_pd, left_on="location_id", right_on="PULocationID", how="left"
).fillna(0)

# Define shared custom color scale red = highest
custom_scale = ["blue","green","yellow","orange","red"]

fig_cold_peak = px.choropleth_mapbox(
    zones_temp_peak_merged,
    geojson=zones_temp_peak_merged.geometry.__geo_interface__,
    locations=zones_temp_peak_merged.index,
    color="log_cold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale=custom_scale,
    title="Heatmap: PM Peak Taxi Pickups — COLD Weather (<5°C)"
)
fig_cold_peak.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_cold_peak.show()

fig_notcold_peak = px.choropleth_mapbox(
    zones_temp_peak_merged,
    geojson=zones_temp_peak_merged.geometry.__geo_interface__,
    locations=zones_temp_peak_merged.index,
    color="log_notcold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale=custom_scale,
    title="Heatmap: PM Peak Taxi Pickups — NON-COLD Weather (>5°C)"
)
fig_notcold_peak.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_notcold_peak.show()

# Calculate the actual min/max of the percent uplift to set a tighter color scale
uplift_values = zones_temp_peak_merged["pct_uplift_cold_vs_notcold"].dropna()
uplift_min = uplift_values.min()
uplift_max = uplift_values.max()

# Set a tighter range with some padding (e.g., use 5th and 95th percentile or actual min/max)
uplift_range = max(abs(uplift_min), abs(uplift_max))
# Round to nearest 5% for cleaner visualization
uplift_range = np.ceil(uplift_range / 5) * 5

fig_uplift_peak = px.choropleth_mapbox(
    zones_temp_peak_merged,
    geojson=zones_temp_peak_merged.geometry.__geo_interface__,
    locations=zones_temp_peak_merged.index,
    color="pct_uplift_cold_vs_notcold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale="RdBu_r",  
    range_color=[-uplift_range, uplift_range],  
    title="Heatmap: Percentage Uplift in Demand — COLD vs Not Cold (PM Peak)"
)
fig_uplift_peak.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_uplift_peak.show()


In [ ]:


from pyspark.sql import functions as F
from pyspark.sql.window import Window
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

pm_peak_hours = [17, 18]  # 5–7 PM

median_temp = combined_df.approxQuantile("temperature_c", [0.5], 0.001)[0]

combined_df_peak = (
    combined_df
    .withColumn("temp_bin",
        F.when(F.col("temperature_c") <= median_temp, "Colder 50%")
         .otherwise("Warmer 50%")
    )
    .withColumn("hour", F.hour("pickup_hour"))
    .filter(F.col("hour").isin(pm_peak_hours))
)
zone_temp = combined_df_peak.groupBy("PULocationID", "temp_bin").agg(
    F.count("*").alias("total_trips"),
    F.countDistinct("pickup_hour").alias("num_hours")
)

zone_temp = zone_temp.withColumn(
    "avg_trips",
    F.when(F.col("num_hours") > 0, F.col("total_trips") / F.col("num_hours")).otherwise(0)
)

zone_temp_pivot = (
    zone_temp.groupBy("PULocationID")
             .pivot("temp_bin", ["Colder 50%", "Warmer 50%"])
             .agg(F.first("avg_trips"))
             .fillna(0)
)

zone_temp_pd = zone_temp_pivot.toPandas()

zone_temp_pd["pct_uplift_cold_vs_warm"] = np.where(
    zone_temp_pd["Warmer 50%"] > 0,
    (zone_temp_pd["Colder 50%"] - zone_temp_pd["Warmer 50%"]) / zone_temp_pd["Warmer 50%"] * 100,
    0
)

# Log scaling for clarity
zone_temp_pd["log_cold"] = np.log1p(zone_temp_pd["Colder 50%"])
zone_temp_pd["log_warm"] = np.log1p(zone_temp_pd["Warmer 50%"])

zones_geo = gpd.read_file("taxi_zones.geojson")
zones_geo["location_id"] = zones_geo["location_id"].astype(int)

zones_temp_merged = zones_geo.merge(
    zone_temp_pd, left_on="location_id", right_on="PULocationID", how="left"
).fillna(0)

# Shared color scale
custom_scale = ["blue","green","yellow","orange","red"]

# =======================
# HEATMAP — Colder 50%
# =======================
fig_cold = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="log_cold",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale=custom_scale,
    title="Heatmap: PM Peak Taxi Pickups — Coldest 50% Temperatures"
)
fig_cold.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_cold.show()

fig_warm = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="log_warm",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale=custom_scale,
    title="Heatmap: PM Peak Taxi Pickups — Warmest 50% Temperatures"
)
fig_warm.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_warm.show()

fig_uplift = px.choropleth_mapbox(
    zones_temp_merged,
    geojson=zones_temp_merged.geometry.__geo_interface__,
    locations=zones_temp_merged.index,
    color="pct_uplift_cold_vs_warm",
    mapbox_style="carto-positron",
    center={"lat": 40.75, "lon": -73.97},
    zoom=9.7,
    opacity=0.85,
    color_continuous_scale=custom_scale,
    title="Heatmap: % Uplift in Taxi Demand — Coldest vs Warmest 50% (PM Peak)"
)
fig_uplift.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_uplift.show()
